## Problema
Precisamos do indicador de tempo desde o ajuizamento do processo, até o julgamento do mesmo.

## Solução
Obter o primeiro movimento relevante - que indica o julgamento do processo - e concatená-lo a tabela de processo.
Posteriormente, calcular o tempo transcorrido entre o ajuizamento do processo e da referida decisão.

In [5]:
import pandas as pd 
import numpy as np

local_dados = 'dados/02_tabelas_transformadas/'

In [6]:
# Calcula movimentos relevantes
def calcular_indicador_tempo_decisao(diretorio):
    """Calcula o indicador de tempo da decisão e salva tab_processo_indicador.csv

    Args:
        diretorio (string): Local onde os arquivos necessários se encontram
    """

    print('calcular_indicador_tempo_decisao... ' + diretorio)

    # Obter tabela processo
    print('Carregando os dados: ' + local_dados +
          diretorio + 'tab_processos_full.csv')
    tabProcesso = pd.read_csv(local_dados + diretorio + 'tab_processos_full.csv',
                              parse_dates=[
                                  'data_ajuizamento', 'data_insercao'],
                              low_memory=False)
    print(tabProcesso.shape)

    # Movimentos relevantes
    print('Carregando os dados: ' + local_dados +
          diretorio + 'tab_movimentos_relevantes.csv')
    tabMovimento = pd.read_csv(
        local_dados + diretorio + 'tab_movimentos_relevantes.csv', parse_dates=['data_movimento'])
    print(tabMovimento.shape)

    # Primeira ocorrência relevante
    print("Calcula primeira ocorrência do movimento relevante")
    tabPriMovimento = tabMovimento.groupby(['dadosBasicos_numero'])[
        'data_movimento'].min().reset_index(name='data_movimento')
    print(tabPriMovimento.shape)
    # Manter dados apenas das primeiras ocorrências dos movimentos relevantes
    tabMovimento = pd.merge(tabMovimento, tabPriMovimento, on=[
                            'dadosBasicos_numero', 'data_movimento'])
    print(tabMovimento.shape)

    # Ligar informação na tabela de Processos
    print('Incluindo data do movimento no processo ')
    tabProcesso = pd.merge(tabProcesso, tabMovimento,
                           how='left', on='dadosBasicos_numero')
    print(tabProcesso.shape)

    # Removendo onde data_ajuizamento > data_movimento
    print('Removendo onde data_ajuizamento > data_movimento')
    tabProcesso = tabProcesso[tabProcesso.data_ajuizamento <
                              tabProcesso.data_movimento]
    print(tabProcesso.shape)

    # Cálculo de Dias e Anos para decisão
    print('Cálculo de Dias e Anos para decisão')
    tabProcesso['dias_para_decisao'] = tabProcesso['data_movimento'] - \
        tabProcesso['data_ajuizamento']
    tabProcesso['anos_para_decisao'] = tabProcesso['dias_para_decisao'] / \
        np.timedelta64(1, 'Y')
    print(tabProcesso.shape)

    # Criar arquivo .CSV
    print('Salvando: ' + local_dados + diretorio + 'tab_processo_indicador.csv')
    tabProcesso.to_csv(local_dados + diretorio +
                       'tab_processo_indicador.csv', index=False)


In [7]:
# Fluxo principal de execução

print("Início cálculo do indicador de tempo da decisão")
calcular_indicador_tempo_decisao('00_justica_federal/processos-trf1/')
calcular_indicador_tempo_decisao('00_justica_federal/processos-trf2/')
calcular_indicador_tempo_decisao('00_justica_federal/processos-trf3/')
calcular_indicador_tempo_decisao('00_justica_federal/processos-trf4/')
calcular_indicador_tempo_decisao('00_justica_federal/processos-trf5/')
print("Fim cálculo do indicador de tempo da decisãor")

Início cálculo do indicador de tempo da decisão
calcular_indicador_tempo_decisao... 00_justica_federal/processos-trf1/
Carregando os dados: dados/02_tabelas_transformadas/00_justica_federal/processos-trf1/tab_processos_full.csv
(19926, 19)
Carregando os dados: dados/02_tabelas_transformadas/00_justica_federal/processos-trf1/tab_movimentos_relevantes.csv
(15114, 7)
Calcula primeira ocorrência do movimento relevante
(10364, 2)
(10364, 7)
Incluindo data do movimento no processo 
(19926, 25)
Removendo onde data_ajuizamento > data_movimento
(9280, 25)
Cálculo de Dias e Anos para decisão
(9280, 27)
Salvando: dados/02_tabelas_transformadas/00_justica_federal/processos-trf1/tab_processo_indicador.csv
calcular_indicador_tempo_decisao... 00_justica_federal/processos-trf2/
Carregando os dados: dados/02_tabelas_transformadas/00_justica_federal/processos-trf2/tab_processos_full.csv
(33542, 19)
Carregando os dados: dados/02_tabelas_transformadas/00_justica_federal/processos-trf2/tab_movimentos_relev